In [1]:
import pandas as pd
import os
import numpy as np
from getpass import getuser
import statsmodels.api as sm



In [2]:
# =============================
# CONFIG
# =============================
USER = getuser()
INPUT_DIR = f"C:/Users/{USER}/Documents/GitHub/tennis-homophily/data/atp/"
INPUT_FILE = os.path.join(INPUT_DIR, "men_matches_with_ranks.xlsx")
OUTPUT_FILE = os.path.join(INPUT_DIR, "win_lose_df.xlsx")
CLEANED_FILE = os.path.join(INPUT_DIR, "men_matches_with_ranks_cleaned.xlsx")
gravity_lang_lookup_dir = os.path.join(INPUT_DIR, "gravity_lang_lookup.xlsx")


In [3]:

# =============================
# Load Dataset
# =============================
print("Loading Grand Slam matches…")
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"Grand Slam file not found: {INPUT_FILE}")

df = pd.read_excel(INPUT_FILE)
print(df.shape)
display(df.head())
df.columns.tolist()


Loading Grand Slam matches…
(1405, 272)


,match_id,tournament,location,date,year,tournament_code,stage,match_duration,winners_set1,winners_set2,...,losers_p2_abs_gap_10y,losers_p2_single_specialist,losers_p2_birthplace-city,losers_p2_birthplace-country,losers_p2_hand,losers_p2_backhand,losers_p2_colonial_legacy,losers_p2_federal_legacy,losers_p2_country_std,losers_p2_iso3
0,0,Australian Open,"Melbourne,Australia",2018-01-15,2018,580,Finals,01:33:00,6,6.0,...,NaN,0.0,Montreal,Canada,Right-Handed,Two-Handed Backhand,Spanish,None,Colombia,COL
1,1,Australian Open,"Melbourne,Australia",2018-01-15,2018,580,Semi-Finals,02:26:00,4,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Australian Open,"Melbourne,Australia",2018-01-15,2018,580,Semi-Finals,01:29:00,7,7.0,...,NaN,0.0,"Camarillo, CA, USA",NaN,Right-Handed,One-Handed Backhand,British,None,United States,USA
3,3,Australian Open,"Melbourne,Australia",2018-01-15,2018,580,Quarter-Finals,01:06:00,6,6.0,...,NaN,0.0,Lahore,Pakistan,Right-Handed,One-Handed Backhand,None,None,Pakistan,PAK
4,4,Australian Open,"Melbourne,Australia",2018-01-15,2018,580,Quarter-Finals,02:40:00,6,6.0,...,NaN,0.0,London,England,Right-Handed,One-Handed Backhand,British,None,Great Britain,GBR


['match_id',
 'tournament',
 'location',
 'date',
 'year',
 'tournament_code',
 'stage',
 'match_duration',
 'winners_set1',
 'winners_set2',
 'winners_set3',
 'winners_set1_tiebreak',
 'winners_set2_tiebreak',
 'winners_set3_tiebreak',
 'losers_set1',
 'losers_set2',
 'losers_set3',
 'losers_set1_tiebreak',
 'losers_set2_tiebreak',
 'losers_set3_tiebreak',
 'winners_set4',
 'winners_set5',
 'losers_set4',
 'losers_set5',
 'winners_set4_tiebreak',
 'losers_set4_tiebreak',
 'losers_set5_tiebreak',
 'winners_p1_name',
 'winners_p1_surname',
 'winners_p1_ranking',
 'winners_p1_status',
 'winners_p2_name',
 'winners_p2_surname',
 'winners_p2_ranking',
 'winners_p2_status',
 'losers_p1_name',
 'losers_p1_surname',
 'losers_p1_ranking',
 'losers_p1_status',
 'losers_p2_name',
 'losers_p2_surname',
 'losers_p2_ranking',
 'losers_p2_status',
 'tourn_key',
 'winners_p1_initial',
 'winners_p2_initial',
 'losers_p1_initial',
 'losers_p2_initial',
 'winners_p1_rank_num',
 'winners_p1_rank',
 'winn

# CEPII data language

In [4]:

# path = f"C:/Users/{USER}/Documents/GitHub/tennis-homophily/data/atp/Gravity_V202211.csv"

# # ISO3 presenti nel dataset match
# iso_set = set(df["winners_p1_iso3"]) | \
#           set(df["winners_p2_iso3"]) | \
#           set(df["losers_p1_iso3"]) | \
#           set(df["losers_p2_iso3"])

# usecols = ["year", "iso3_o", "iso3_d", "comlang_off", "comlang_ethno"]

# dtypes = {
#     "year": "int16",
#     "iso3_o": "string",
#     "iso3_d": "string",
#     "comlang_off": "Int8",
#     "comlang_ethno": "Int8",
# }

# chunks = []

# for chunk in pd.read_csv(path, usecols=usecols, dtype=dtypes, chunksize=300_000):

#     # filtro anni
#     chunk = chunk[chunk["year"].between(2018, 2023)]
#     if chunk.empty:
#         continue

#     # filtro paesi rilevanti
#     chunk = chunk[
#         chunk["iso3_o"].isin(iso_set) &
#         chunk["iso3_d"].isin(iso_set)
#     ]

#     if not chunk.empty:
#         chunks.append(chunk)

# gravity_lang = pd.concat(chunks, ignore_index=True)

# gravity_lang.shape


In [5]:
# # Creazione dataset simmetrico (iso3_o, iso3_d) <-> (iso3_d, iso3_o)
# gravity_lang_sym = pd.concat(
#     [
#         gravity_lang,
#         gravity_lang.rename(columns={"iso3_o":"iso3_d","iso3_d":"iso3_o"})
#     ],
#     ignore_index=True
# )

# gravity_lang_sym = gravity_lang_sym.drop_duplicates(
#     subset=["year","iso3_o","iso3_d"]
# )


In [6]:
# # symmetry: same nationality -->  same language
# same = gravity_lang_sym["iso3_o"] == gravity_lang_sym["iso3_d"]
# gravity_lang_sym.loc[same, ["comlang_off","comlang_ethno"]] = 1
# gravity_lang_sym


In [7]:
# # make dataset lookup: (iso3_o, iso3_d) -> (comlang_off, comlang_ethno)
# gravity_lang_lookup = (
#     gravity_lang_sym
#     .groupby(["iso3_o","iso3_d"], as_index=False)[
#         ["comlang_off","comlang_ethno"]
#     ]
#     .max()
# )
# gravity_lang_lookup


In [8]:
# # =============================
# # Save
# # =============================
# print(f"Saving to {gravity_lang_lookup_dir} …")
# with pd.ExcelWriter(gravity_lang_lookup_dir, engine="xlsxwriter") as xlw:
#     gravity_lang_lookup.to_excel(xlw, index=False, sheet_name="main_sheet")

# print("Done.")


In [9]:
gravity_lang_lookup = pd.read_excel(gravity_lang_lookup_dir, sheet_name="main_sheet")

In [10]:
# merge winners
df = df.merge(
    gravity_lang_lookup,
    left_on=["winners_p1_iso3","winners_p2_iso3"],
    right_on=["iso3_o","iso3_d"],
    how="left"
)

df = df.rename(columns={
    "comlang_off":"winners_same_language",
    "comlang_ethno":"winners_linguistic_proximity"
}).drop(columns=["iso3_o","iso3_d"])

# merge losers
df = df.merge(
    gravity_lang_lookup,
    left_on=["losers_p1_iso3","losers_p2_iso3"],
    right_on=["iso3_o","iso3_d"],
    how="left"
)

df = df.rename(columns={
    "comlang_off":"losers_same_language",
    "comlang_ethno":"losers_linguistic_proximity"
}).drop(columns=["iso3_o","iso3_d"])



  # Filter dataset

In [11]:
# Rimuovi l'anno 2020 dal dataset, COVID time, Wimblendon non si è giocato
df = df[df['year'] != 2020].copy()

# Controllo rapido
print(df['year'].value_counts())

2018    262
2019    250
2021    249
2022    249
2023    238
Name: year, dtype: int64


# Create new variables

In [12]:
# Create surface variable
surface_map = {
    "Australian Open": "Hard",
    "US Open": "Hard",
    "Roland Garros": "Clay",
    "Wimbledon": "Grass"
}

df['surface'] = df['tournament'].map(surface_map)


In [13]:
# Normalize stage names
df['stage_clean'] = df['stage'].str.lower().str.strip()

stage_order = {
    "finals": 7,
    "semi-finals": 6,
    "quarter-finals": 5,
    "round of 16": 4,
    "round of 32": 3,
    "round of 64": 2,
    "2nd round qualifying": 1,
    "1st round qualifying": 0
}

df['stage_code'] = df['stage_clean'].map(stage_order)
df['stage_code'].unique()

array([7, 6, 5, 4, 3, 2, 1, 0], dtype=int64)

In [14]:
# --- Physical Homophily --- #

# --- hand ---

df['same_hand_winners'] = (df['winners_p1_hand'] == df['winners_p2_hand']).astype(int)
df['same_hand_losers']  = (df['losers_p1_hand']  == df['losers_p2_hand']).astype(int)

# --- backhand ---

df['same_backhand_winners'] = (df['winners_p1_backhand'] == df['winners_p2_backhand']).astype(int)
df['same_backhand_losers']  = (df['losers_p1_backhand']  == df['losers_p2_backhand']).astype(int)

# --- height ---
df['height_diff_winners'] = (df['winners_p1_height-cm'] - df['winners_p2_height-cm']).abs()
df['height_diff_losers']  = (df['losers_p1_height-cm']  - df['losers_p2_height-cm']).abs()

# --- weight ---
df['weight_diff_winners'] = (df['winners_p1_weight-kg'] - df['winners_p2_weight-kg']).abs()
df['weight_diff_losers']  = (df['losers_p1_weight-kg']  - df['losers_p2_weight-kg']).abs()

# --- experience ---

df['experience_diff_winners'] = (df['winners_p1_experience_double'] - df['winners_p2_experience_double']).abs()
df['experience_diff_losers']  = (df['losers_p1_experience_double']  - df['losers_p2_experience_double']).abs()

# --- rank/ability ---

df['rank_diff_winners'] = (df['winners_p1_rank_num'] - df['winners_p2_rank_num']).abs()
df['rank_mean_winners'] = (df['winners_p1_rank_num'] + df['winners_p2_rank_num'])/2
df['rank_diff_losers']  = (df['losers_p1_rank_num']  - df['losers_p2_rank_num']).abs()
df['rank_mean_losers']  = (df['losers_p1_rank_num']  + df['losers_p2_rank_num'])/2

df["single_rank_diff_winners"] = (df['winners_p1_singles_career_high_rank_num'] - df['winners_p2_singles_career_high_rank_num']).abs()
df["single_rank_diff_losers"]  = (df['losers_p1_singles_career_high_rank_num']  - df['losers_p2_singles_career_high_rank_num']).abs()

In [15]:

# --- Social Homophily --- #

# --- nationality ---
df['same_country_winners'] = (df['winners_p1_country'] == df['winners_p2_country']).astype(int)
df['same_country_losers']  = (df['losers_p1_country']  == df['losers_p2_country']).astype(int)

# --- coach ---
df['same_coach_winners'] = (
    (df['winners_p1_coach'].notna()) &
    (df['winners_p2_coach'].notna()) &
    (df['winners_p1_coach'] == df['winners_p2_coach'])
).astype(int)

df['same_coach_losers'] = (
    (df['losers_p1_coach'].notna()) &
    (df['losers_p2_coach'].notna()) &
    (df['losers_p1_coach'] == df['losers_p2_coach'])
).astype(int)


In [16]:

# Differenza assoluta nel win ratio YTD e Career
df['wl_ytd_diff_winners'] = (df['winners_p1_doubles_win_ratio-ytd'] - df['winners_p2_doubles_win_ratio-ytd']).abs()
df['wl_ytd_diff_losers'] = (df['losers_p1_doubles_win_ratio-ytd']  - df['losers_p2_doubles_win_ratio-ytd']).abs()

df['wl_career_diff_winners'] = (df['winners_p1_doubles_win_ratio-career'] - df['winners_p2_doubles_win_ratio-career']).abs()
df['wl_career_diff_losers']  = (df['losers_p1_doubles_win_ratio-career']  - df['losers_p2_doubles_win_ratio-career']).abs()

# Differenza in titoli (YTD e Career)
df['titles_ytd_diff_winners']    = (df['winners_p1_doubles_titles-ytd'] - df['winners_p2_doubles_titles-ytd']).abs()
df['titles_ytd_diff_losers']     = (df['losers_p1_doubles_titles-ytd'] - df['losers_p2_doubles_titles-ytd']).abs()

df['titles_career_diff_winners'] = (df['winners_p1_doubles_titles-career'] - df['winners_p2_doubles_titles-career']).abs()
df['titles_career_diff_losers']  = (df['losers_p1_doubles_titles-career'] - df['losers_p2_doubles_titles-career']).abs()

# single specialists
df['single_specialist_winners'] = (
  (df['winners_p1_single_specialist'] == 1) & (df['winners_p2_single_specialist'] == 1)
).astype(int)

df['single_specialist_losers']  = (
  (df['losers_p1_single_specialist'] == 1) & (df['losers_p2_single_specialist'] == 1)
).astype(int)

# dataset: one row for winners, one row for losers

In [17]:
team_rows = []

for i, row in df.iterrows():

    # --------------------------
    # WINNERS ROW
    # --------------------------
    team_rows.append({
        'match_id': row['match_id'],
        'team': 'winners',
        'won': 1,

        # Z-differences
        'height_diff': row['height_diff_winners'],
        'weight_diff': row['weight_diff_winners'],
        'experience_diff': row['experience_diff_winners'],

        # Player experience
        'experience_p1': row['winners_p1_experience_double'],
        'experience_p2': row['winners_p2_experience_double'],

        # single specialists
        'single_specialist': row['single_specialist_winners'],
        'single_specialist_p1': row['winners_p1_single_specialist'],
        'single_specialist_p2': row['winners_p2_single_specialist'],

        # Rank
        'rank_p1': row['winners_p1_rank_num'],
        'rank_p2': row['winners_p2_rank_num'],
        'rank_diff': row['rank_diff_winners'],
        "rank_single_p1": row['winners_p1_singles_career_high_rank_num'],
        "rank_single_p2": row['winners_p2_singles_career_high_rank_num'],
        'rank_diff_single': row['single_rank_diff_winners'],

        # W/L career ratio
        'wl_career_diff': row['wl_career_diff_winners'],  
        'wl_career_p1': row['winners_p1_doubles_win_ratio-career'],
        'wl_career_p2': row['winners_p2_doubles_win_ratio-career'],

        # W/L YTD ratio
        'wl_ytd_diff': row['wl_ytd_diff_winners'],
        'wl_ytd_p1': row['winners_p1_doubles_win_ratio-ytd'],
        'wl_ytd_p2': row['winners_p2_doubles_win_ratio-ytd'],

        # Titles (career)
        'titles_p1': row['winners_p1_doubles_titles-career'],
        'titles_p2': row['winners_p2_doubles_titles-career'],
        'titles_career_diff': row['titles_career_diff_winners'],

        # Homophily components
        'same_country': row['same_country_winners'],
        'same_hand': row['same_hand_winners'],
        'same_backhand': row['same_backhand_winners'],
        'same_coach': row['same_coach_winners'],
        'same_language': row['winners_same_language'],
        'linguistic_proximity': row['winners_linguistic_proximity'],

        # Match meta
        'year': row['year'],
        'tournament': row['tournament'],
        'stage': row['stage_code'],
        'surface': row['surface']
    })


    # --------------------------
    # LOSERS ROW
    # --------------------------
    team_rows.append({
        'match_id': row['match_id'],
        'team': 'losers',
        'won': 0,

        # Z-differences
        'height_diff': row['height_diff_losers'],
        'weight_diff': row['weight_diff_losers'],
        'experience_diff': row['experience_diff_losers'],

        # Player experience
        'experience_p1': row['losers_p1_experience_double'],
        'experience_p2': row['losers_p2_experience_double'],

        # Rank
        'rank_p1': row['losers_p1_rank_num'],
        'rank_p2': row['losers_p2_rank_num'],
        'rank_diff': row['rank_diff_losers'],
        "rank_single_p1": row['losers_p1_singles_career_high_rank_num'],
        "rank_single_p2": row['losers_p2_singles_career_high_rank_num'],
        'rank_diff_single': row['single_rank_diff_losers'],

        # single specialists
        'single_specialist': row['single_specialist_losers'],
        'single_specialist_p1': row['losers_p1_single_specialist'],
        'single_specialist_p2': row['losers_p2_single_specialist'],

        # W/L career ratio
        'wl_career_diff': row['wl_career_diff_losers'],   
        'wl_career_p1': row['losers_p1_doubles_win_ratio-career'],
        'wl_career_p2': row['losers_p2_doubles_win_ratio-career'],

        # W/L YTD ratio
        'wl_ytd_diff': row['wl_ytd_diff_losers'],
        'wl_ytd_p1': row['losers_p1_doubles_win_ratio-ytd'],
        'wl_ytd_p2': row['losers_p2_doubles_win_ratio-ytd'],

        # Titles
        'titles_p1': row['losers_p1_doubles_titles-career'],
        'titles_p2': row['losers_p2_doubles_titles-career'],
        'titles_career_diff': row['titles_career_diff_losers'],

        # Homophily components
        'same_country': row['same_country_losers'],
        'same_hand': row['same_hand_losers'],
        'same_backhand': row['same_backhand_losers'],
        'same_coach': row['same_coach_losers'],
        'same_language': row['losers_same_language'],
        'linguistic_proximity': row['losers_linguistic_proximity'],

        # Match meta
        'year': row['year'],
        'tournament': row['tournament'],
        'stage': row['stage_code'],
        'surface': row['surface']
    })

# Build the final team-level dataframe
team_df = pd.DataFrame(team_rows)


In [18]:
# Team-level ability (media/somma)
team_df['rank_team'] = team_df[['rank_p1', 'rank_p2']].mean(axis=1)
team_df['rank_team_best'] = team_df[['rank_p1', 'rank_p2']].min(axis=1)
team_df['rank_single_team'] = team_df[['rank_single_p1', 'rank_single_p2']].mean(axis=1)
team_df['rank_single_best'] = team_df[['rank_single_p1', 'rank_single_p2']].min(axis=1)
team_df['experience_team'] = (team_df['experience_p1'] + team_df['experience_p2']) / 2
team_df['wl_career_team'] = team_df['wl_career_p1'] + team_df['wl_career_p2']
team_df['wl_ytd_team'] = team_df['wl_ytd_p1'] + team_df['wl_ytd_p2']
team_df['titles_team'] = team_df['titles_p1'] + team_df['titles_p2']
team_df['wl_ratio_team_career'] = team_df['wl_career_p1'] + team_df['wl_career_p2']
team_df['wl_ratio_team_ytd']    = team_df['wl_ytd_p1']    + team_df['wl_ytd_p2']


# save datasets

In [19]:
# =============================
# Save
# =============================
print(f"Saving to {OUTPUT_FILE} …")
with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as xlw:
    team_df.to_excel(xlw, index=False, sheet_name="players_list")

print("Done.")


Saving to C:/Users/aldi/Documents/GitHub/tennis-homophily/data/atp/win_lose_df.xlsx …
Done.


In [20]:
# =============================
# Save
# =============================
print(f"Saving to {CLEANED_FILE} …")
with pd.ExcelWriter(CLEANED_FILE, engine="xlsxwriter") as xlw:
    df.to_excel(xlw, index=False, sheet_name="players_list")

print("Done.")


Saving to C:/Users/aldi/Documents/GitHub/tennis-homophily/data/atp/men_matches_with_ranks_cleaned.xlsx …
Done.


 # some stats

## single specialist + best in pair

In [21]:
### WINNERS ###

w_p1_single = df["winners_p1_single_specialist"].eq(1)
w_p2_single = df["winners_p2_single_specialist"].eq(1)

w_p1_best = df["winners_p1_rank_num"] < df["winners_p2_rank_num"]
w_p2_best = df["winners_p2_rank_num"] < df["winners_p1_rank_num"]

df["winner_specialist_best"] = (
    (w_p1_single & w_p1_best) |
    (w_p2_single & w_p2_best)
).astype(int)


### LOSERS ###

l_p1_single = df["losers_p1_single_specialist"].eq(1)
l_p2_single = df["losers_p2_single_specialist"].eq(1)

l_p1_best = df["losers_p1_rank_num"] < df["losers_p2_rank_num"]
l_p2_best = df["losers_p2_rank_num"] < df["losers_p1_rank_num"]

df["loser_specialist_best"] = (
    (l_p1_single & l_p1_best) |
    (l_p2_single & l_p2_best)
).astype(int)


In [22]:
df["structure_diff"] = df["winner_specialist_best"] - df["loser_specialist_best"]

structure_counts = df["structure_diff"].value_counts()
print("Structure comparison:")
print(structure_counts)

advantage_wins = (df["structure_diff"] == 1).sum()
advantage_losses = (df["structure_diff"] == -1).sum()

if (advantage_wins + advantage_losses) > 0:
    advantage_rate = advantage_wins / (advantage_wins + advantage_losses)
else:
    advantage_rate = np.nan

print("\nWin rate when structure differs:", round(advantage_rate,4))


Structure comparison:
 0    1201
-1      25
 1      22
Name: structure_diff, dtype: int64

Win rate when structure differs: 0.4681


When winning team has a “specialist + best player” structure, that team wins 46.8% of the time. (22/(25+22))

## does the team with the best current double ranked player win more often?

In [23]:
df["winner_best_rank"] = df[["winners_p1_rank_num","winners_p2_rank_num"]].min(axis=1)
df["winner_worst_rank"] = df[["winners_p1_rank_num","winners_p2_rank_num"]].max(axis=1)
df["loser_best_rank"]  = df[["losers_p1_rank_num","losers_p2_rank_num"]].min(axis=1)
df["loser_worst_rank"]  = df[["losers_p1_rank_num","losers_p2_rank_num"]].max(axis=1)


In [24]:
df["winner_has_best_current_rank"] = (
    df["winner_best_rank"] < df["loser_best_rank"]
).astype(int)

current_rank_wins = df["winner_has_best_current_rank"].sum()
total_valid = df["winner_has_best_current_rank"].notna().sum()

print("\nWins by team with best CURRENT rank:", current_rank_wins)
print("Share:", round(current_rank_wins / total_valid,4))



Wins by team with best CURRENT rank: 496
Share: 0.3974


# does the team with the best and worst players win more often than the one with the two middle? 1-4 vs 2-3?

In [25]:

# --- 4 ranks in the match ---
# best = min rank number (rank 1 is best), worst = max rank number
df["match_best_rank"]  = df[["winner_best_rank", "loser_best_rank"]].min(axis=1)
df["match_worst_rank"] = df[["winner_worst_rank", "loser_worst_rank"]].max(axis=1)

# Winner has the best player in the match?
df["winner_has_match_best"] = (df["winner_best_rank"] == df["match_best_rank"])

# Winner has the worst player in the match?
df["winner_has_match_worst"] = (df["winner_worst_rank"] == df["match_worst_rank"])

# "1+4 team" is the team that has BOTH the best and the worst player
df["winner_is_14_team"] = (df["winner_has_match_best"] & df["winner_has_match_worst"]).astype(int)

# Identify matches that are truly (1+4) vs (2+3):
# i.e., one team has best+worst and the other team has the two middle players.
# Equivalent condition: one side has both extremes.
df["is_14_vs_23"] = (
    (df["winner_has_match_best"] & df["winner_has_match_worst"]) |
    ((df["loser_best_rank"] == df["match_best_rank"]) & (df["loser_worst_rank"] == df["match_worst_rank"]))
)

# Keep only valid rows with no missing ranks
valid = df.dropna(subset=[
    "winner_best_rank","winner_worst_rank","loser_best_rank","loser_worst_rank"
]).copy()

valid_14_23 = valid[valid["is_14_vs_23"]].copy()

# --- Paper-style percentages ---
share_14_23 = len(valid_14_23) / len(valid)  # how often the match is 1+4 vs 2+3
winrate_14 = valid_14_23["winner_is_14_team"].mean()  # in those matches, does 1+4 win?

print("Matches with complete ranks:", len(valid))
print("1+4 vs 2+3 matches:", len(valid_14_23))
print("Share of matches that are 1+4 vs 2+3:", round(share_14_23, 4))
print("Win rate of 1+4 team in those matches:", round(winrate_14, 4))


Matches with complete ranks: 769
1+4 vs 2+3 matches: 61
Share of matches that are 1+4 vs 2+3: 0.0793
Win rate of 1+4 team in those matches: 0.459


In [26]:
valid["winner_has_match_best"]  = (valid["winner_best_rank"]  == valid["match_best_rank"])
valid["winner_has_match_worst"] = (valid["winner_worst_rank"] == valid["match_worst_rank"])

print("\nWinner has the BEST player (share):", round(valid["winner_has_match_best"].mean(), 4))
print("Winner has the WORST player (share):", round(valid["winner_has_match_worst"].mean(), 4))



Winner has the BEST player (share): 0.645
Winner has the WORST player (share): 0.3485


In [27]:
# loser_is_14_team should be the complement inside valid_14_23
valid_14_23["loser_is_14_team"] = 1 - valid_14_23["winner_is_14_team"]

# Check: in all such matches, exactly one side is 1+4
check = (valid_14_23["winner_is_14_team"] + valid_14_23["loser_is_14_team"]).value_counts()
print("\nCheck (should be all 1s):")
print(check)



Check (should be all 1s):
1    61
dtype: int64


## does the magnitude of the rank difference matter?

In [28]:
df["best_rank_diff"] = df["loser_best_rank"] - df["winner_best_rank"]


In [29]:
df["best_rank_diff"].min(), df["best_rank_diff"].max()


(-89.0, 98.0)

In [30]:
df["abs_rank_diff"] = df["best_rank_diff"].abs()

df["rank_gap_group"] = pd.cut(
    df["abs_rank_diff"],
    bins=[0, 5, 10, 20, 40, 60, 99]
)

gap_table = (
    df[df["best_rank_diff"] != 0]
    .assign(better_team_wins = df["best_rank_diff"] > 0)
    .groupby("rank_gap_group")["better_team_wins"]
    .mean()
    .to_frame("Prob better-ranked team wins")
)

gap_table


,Prob better-ranked team wins
rank_gap_group,
"(0, 5]",0.573529
"(5, 10]",0.583333
"(10, 20]",0.627907
"(20, 40]",0.616071
"(40, 60]",0.670732
"(60, 99]",0.767857


## does the team with the best single ranked player win more often?

In [31]:
df["winner_best_career"] = df[
    ["winners_p1_singles_career_high_rank_num",
     "winners_p2_singles_career_high_rank_num"]
].min(axis=1)

df["loser_best_career"] = df[
    ["losers_p1_singles_career_high_rank_num",
     "losers_p2_singles_career_high_rank_num"]
].min(axis=1)


In [32]:
df["winner_has_best_career_rank"] = (
    df["winner_best_career"] < df["loser_best_career"]
).astype(int)

career_rank_wins = df["winner_has_best_career_rank"].sum()
total_valid_career = df["winner_has_best_career_rank"].notna().sum()

print("\nWins by team with best CAREER-HIGH rank:", career_rank_wins)
print("Share:", round(career_rank_wins / total_valid_career,4))



Wins by team with best CAREER-HIGH rank: 409
Share: 0.3277


## remove all single specialist, include only players which do not have a single best or those whose single best is older than 5 years 

In [36]:
top_cols = [
    "winners_p1_top100_within_1y",
    "winners_p1_top100_within_5y",
    "winners_p2_top100_within_1y",
    "winners_p2_top100_within_5y",
    "losers_p1_top100_within_1y",
    "losers_p1_top100_within_5y",
    "losers_p2_top100_within_1y",
    "losers_p2_top100_within_5y"
]

df[top_cols] = df[top_cols].fillna(0)

In [37]:
df["has_recent_single_elite"] = (
    df[top_cols].sum(axis=1) > 0
).astype(int)

In [38]:
df_no_recent_single = df[df["has_recent_single_elite"] == 0].copy()

In [39]:
print("Original sample:", len(df))
print("Restricted sample:", len(df_no_recent_single))
print("Share removed:", round(1 - len(df_no_recent_single)/len(df), 4))

Original sample: 1248
Restricted sample: 978
Share removed: 0.2163


In [40]:
df_sub = df_no_recent_single.copy()

# Best-ranked player in each team
df_sub["winner_best_rank"] = df_sub[["winners_p1_rank_num",
                                     "winners_p2_rank_num"]].min(axis=1)

df_sub["loser_best_rank"] = df_sub[["losers_p1_rank_num",
                                    "losers_p2_rank_num"]].min(axis=1)

In [41]:
df_sub["winner_has_best_player"] = (
    df_sub["winner_best_rank"] < df_sub["loser_best_rank"]
).astype(int)

In [42]:
total_valid = df_sub["winner_has_best_player"].notna().sum()
wins_best_player = df_sub["winner_has_best_player"].sum()

share = wins_best_player / total_valid

print("Matches in restricted sample:", total_valid)
print("Wins by team with best-ranked player:", wins_best_player)
print("Share:", round(share, 4))

Matches in restricted sample: 978
Wins by team with best-ranked player: 364
Share: 0.3722


# linguistic

In [33]:
cols = [
    "winners_same_language",
    "winners_linguistic_proximity",
    "losers_same_language",
    "losers_linguistic_proximity"
]

for c in cols:
    df[c] = df[c].fillna(0).astype(int)


In [34]:
perc_table = (
    df[cols]
    .mean()
    .mul(100)
    .round(2)
    .to_frame(name="Percentage (%)")
)

perc_table


,Percentage (%)
winners_same_language,38.94
winners_linguistic_proximity,39.34
losers_same_language,25.56
losers_linguistic_proximity,26.12
